In [1]:
import keras,os
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D,Dropout, BatchNormalization
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from keras import optimizers
from keras.layers.core import Lambda
from keras import backend as L
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# train_path = 'char_output_extended/train'
# test_path = 'char_output_extended/test'

# train_gendata = ImageDataGenerator(rescale = 1./255,validation_split = 0.15)

# valid_gendata = ImageDataGenerator(rescale = 1./255,validation_split = 0.15)

# test_gendata  = ImageDataGenerator(rescale = 1./255)
# training_dataset  = train_gendata.flow_from_directory(directory = train_path,target_size = (128,128),
#                                                    class_mode = 'categorical',subset = 'training',batch_size = 32)
# validation_dataset = valid_gendata.flow_from_directory(directory = train_path,target_size = (128,128),
#                                                   class_mode = 'categorical',subset = 'validation',batch_size = 32)
# testing_dataset = test_gendata.flow_from_directory(directory = test_path,target_size = (128,128),
#                                                 class_mode = 'categorical',batch_size = 32)


In [3]:
train_path = 'char_balanced/train'
test_path = 'char_balanced/test'

train_gendata = ImageDataGenerator(rescale=1./255, validation_split=0.15)
valid_gendata = ImageDataGenerator(rescale=1./255, validation_split=0.15)
test_gendata = ImageDataGenerator(rescale=1./255)

training_dataset = train_gendata.flow_from_directory(directory=train_path, target_size=(128,128),
                                                    class_mode='sparse', subset='training', batch_size=32)
validation_dataset = valid_gendata.flow_from_directory(directory=train_path, target_size=(128,128),
                                                       class_mode='sparse', subset='validation', batch_size=32)
testing_dataset = test_gendata.flow_from_directory(directory=test_path, target_size=(128,128),
                                                  class_mode='sparse', batch_size=32)

Found 3072 images belonging to 48 classes.
Found 528 images belonging to 48 classes.
Found 4744 images belonging to 48 classes.


In [4]:
# CUSTOM MODEL
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(48, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
palm_model = model.fit(training_dataset, validation_data=validation_dataset, epochs=40, steps_per_epoch=5, verbose=1)


Epoch 1/40


2023-06-04 15:02:44.449918: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


5/5 [==============================] - 2s 353ms/step - loss: 4.4262 - accuracy: 0.0125 - val_loss: 3.8774 - val_accuracy: 0.0208
Epoch 2/40
5/5 [==============================] - 1s 329ms/step - loss: 3.8775 - accuracy: 0.0250 - val_loss: 3.8696 - val_accuracy: 0.0208
Epoch 3/40
5/5 [==============================] - 1s 326ms/step - loss: 3.8752 - accuracy: 0.0188 - val_loss: 3.8620 - val_accuracy: 0.0606
Epoch 4/40
5/5 [==============================] - 1s 327ms/step - loss: 3.8543 - accuracy: 0.0437 - val_loss: 3.8581 - val_accuracy: 0.0284
Epoch 5/40
5/5 [==============================] - 1s 322ms/step - loss: 3.8738 - accuracy: 0.0250 - val_loss: 3.8403 - val_accuracy: 0.0227
Epoch 6/40
5/5 [==============================] - 1s 323ms/step - loss: 3.8543 - accuracy: 0.0125 - val_loss: 3.8173 - val_accuracy: 0.0852
Epoch 7/40
5/5 [==============================] - 1s 328ms/step - loss: 3.8112 - accuracy: 0.0562 - val_loss: 3.7554 - val_accuracy: 0.0890
Epoch 8/40
5/5 [===============

In [5]:
plt.plot(palm_model.history['loss'])
plt.plot(palm_model.history['val_loss'])
plt.title('Checking Fit')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image


# Define the class labels
class_labels = training_dataset.class_indices
class_labels = {v: k for k, v in class_labels.items()}

# Function to predict class labels for new images
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0  # Normalize the image

    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)

    return class_labels[predicted_class]

# Example usage
image_path = '/Users/sougandhthatavarthi/Desktop/University/SEM6/DLSIP/Team21_project/whiteblack/split/folder_1/MaI12_Page53.jpg_char_89.jpg'  # Replace 'path_to_image.jpg' with the actual path to your test image
predicted_label = predict_image(image_path)
print('Predicted class label:', predicted_label)


1/1 [==============================] - 0s 14ms/step
Predicted class label: 3346


In [7]:
# import numpy as np
# from tensorflow.keras.preprocessing import image


# # Load a new image to predict its class
# img_path = 'balanced_dataset/test2.jpg.png'
# img = image.load_img(img_path, target_size=(128, 128))
# img_array = image.img_to_array(img)
# img_array = np.expand_dims(img_array, axis=0)
# img_array /= 255.0

# # Make a prediction
# predictions = model.predict(img_array)
# predicted_class = np.argmax(predictions[0])

# print(predicted_class)

# class_indices = training_dataset.class_indices
# inverse_map = {v: k for k, v in class_indices.items()}
# predicted_label = inverse_map[predicted_class]

# print(predicted_label)

In [7]:
model.save('customcnn_alpha.h5')